In [137]:
import numpy as np
from itertools import combinations

def calculate_path_dependency(X_clean, path, denominators):
    """Calcula la dependencia de un camino específico."""
    # Inicializamos con el primer flujo
    flujo_actual = X_clean[path[0], path[1]]
    
    # Para cada nodo intermedio, calculamos qué proporción del flujo continúa
    for i in range(1, len(path)-1):
        nodo_actual = path[i]
        # Si el denominador es 0, no hay flujo que pueda pasar
        if denominators[nodo_actual] == 0:
            return 0
        # Calculamos qué proporción del flujo sigue al siguiente nodo
        proporcion = X_clean[path[i], path[i+1]] / denominators[nodo_actual]
        flujo_actual = flujo_actual * proporcion
    
    # Normalizamos por el denominador del país destino
    return flujo_actual / denominators[path[-1]] if denominators[path[-1]] > 0 else 0


def calculate_all_dependencies(X, convergence_threshold=0.01, max_possible_length=5, threshold_pct=0.1):
    """Calcula todas las dependencias entre países."""
    n = X.shape[0]

    # Preprocesamiento
    total_trade = np.sum(X)
    threshold = total_trade * (threshold_pct / 100)
    X_clean = np.where(X < threshold, 0, X)
    denominators = np.sum(X, axis=0)

    results = {
        'dependencies': [],
        'top_dependencies': [],
        'avg_dependencies': np.zeros(n),
        'length_distribution': np.zeros(max_possible_length),
    }

    for i in range(n):  # País importador
        total_dep = 0
        num_deps = 0

        for j in range(n):  # País exportador
            if i != j:
                current_total = 0
                indirect_total = 0
                dependencies_by_length = {}

                # Dependencia directa
                direct_dependency = X_clean[j, i] / denominators[i] if denominators[i] > 0 else 0
                dependencies_by_length[1] = direct_dependency
                current_total = direct_dependency

                # Dependencias indirectas
                for length in range(2, max_possible_length + 1):
                    DI_ij_l = 0
                    middle_countries = [k for k in range(n) if k != i and k != j]

                    for intermediaries in combinations(middle_countries, length - 1):
                        path = (j,) + intermediaries + (i,)  # Orden corregido: exportador -> importador
                        fuerza_camino = calculate_path_dependency(X_clean, path, denominators)  # Llamada CORRECTA
                        DI_ij_l += fuerza_camino

                    dependencies_by_length[length] = DI_ij_l
                    indirect_total += DI_ij_l

                    prev_total = current_total
                    current_total = direct_dependency + indirect_total

                    # Convergencia
                    if length > 1 and abs(current_total - prev_total) < convergence_threshold:
                        results['length_distribution'][length - 1] += 1
                        break

                # La dependencia total es la suma de la directa y la indirecta
                total_dependency = direct_dependency + indirect_total


                # Almacenar resultado (usando la dependencia indirecta *ponderada*)
                result = {
                    'importador': i,
                    'exportador': j,
                    'dependencia_directa': direct_dependency,
                    'dependencia_indirecta': indirect_total,  # Valor corregido
                    'dependencia_total': total_dependency,  # Valor corregido
                    'longitud_optima': length,
                    'dependencias_por_longitud': dependencies_by_length
                }

                results['dependencies'].append(result)
                results['top_dependencies'].append(
                    (i, j, direct_dependency, indirect_total, total_dependency, length))  # Valores corregidos
                total_dep += total_dependency
                num_deps += 1

        results['avg_dependencies'][i] = total_dep / num_deps if num_deps > 0 else 0

    # Ordenar top dependencies
    results['top_dependencies'].sort(key=lambda x: x[4], reverse=True)
    results['top_dependencies'] = results['top_dependencies'][:20]

    return results

def analyze_dependencies(X, country_names=None):
    """Analiza las dependencias comerciales y muestra los resultados."""
    results = calculate_all_dependencies(X)

    print("\nTop 10 dependencias más fuertes:")
    print("Importador | Exportador | Directa   | Indirecta | Total     | Long. Óptima")
    print("-" * 75)
    
    for i, j, dep_dir, dep_ind, dep_tot, length in results['top_dependencies']:
        imp_name = country_names[i] if country_names else f"País {i}"
        exp_name = country_names[j] if country_names else f"País {j}"
        print(f"{imp_name:10} | {exp_name:10} | {dep_dir:8.4f} | {dep_ind:9.4f} | {dep_tot:8.4f} | {length:12d}")

    print("\nDependencia promedio por país importador:")
    for i, avg_dep in enumerate(results['avg_dependencies']):
        name = country_names[i] if country_names else f"País {i}"
        print(f"{name}: {avg_dep:.4f}")

    return results

if __name__ == "__main__":

    # Matriz 20x20 con más ceros y valores destacados
    X = np.array([
        [100,  80,   0,   0,   0,   0,   0,   0,   0,  60,   0,   0,   0,   0,  40,   0,   0,   0,   0,   0],  # USA
        [90,   90,  45,  70,   0,   0,  50,   0,   0,   0,  35,   0,   0,   0,   0,  30,   0,   0,   0,   0],  # China
        [0,    35, 100,   0,  40,  65,   0,  55,   0,   0,   0,   0,   0,  45,   0,   0,  50,  40,   0,   0],  # Alemania
        [0,    75,   0,  85,   0,   0,   0,   0,   0,   0,  60,   0,   0,   0,   0,   0,   0,   0,   0,   0],  # Japón
        [0,     0,  35,   0,  95,  40,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  35,   0,   0,   0],  # Reino Unido
        [0,     0,  55,   0,  45,  90,   0,  50,   0,   0,   0,   0,   0,  40,   0,   0,  45,   0,   0,   0],  # Francia
        [0,    45,   0,   0,   0,   0,  80,   0,   0,   0,   0,   0,   0,   0,   0,  35,   0,   0,   0,   0],  # India
        [0,     0,  50,   0,   0,  45,   0,  85,   0,   0,   0,   0,   0,  35,   0,   0,   0,   0,   0,   0],  # Italia
        [0,     0,   0,   0,   0,   0,   0,   0,  75,   0,   0,   0,  40,   0,  35,   0,   0,   0,   0,   0],  # Brasil
        [70,    0,   0,   0,   0,   0,   0,   0,   0,  80,   0,   0,   0,   0,  45,   0,   0,   0,   0,   0],  # Canadá
        [0,    40,   0,  65,   0,   0,   0,   0,   0,   0,  85,   0,   0,   0,   0,   0,   0,   0,   0,   0],  # Corea Sur
        [0,     0,  80,   0,   0,   0,   0,   0,   0,   0,   0,  90,   0,   0,   0,   0,   0,   0,  70,   0],  # Rusia
        [0,     0,   0,   0,   0,   0,   0,   0,  35,   0,   0,   0,  80,   0,   0,  45,   0,   0,   0,   0],  # Australia
        [0,     0,  40,   0,   0,  35,   0,  30,   0,   0,   0,   0,   0,  85,   0,   0,   0,   0,   0,   0],  # España
        [55,    0,   0,   0,   0,   0,   0,   0,  30,  40,   0,   0,   0,   0,  75,   0,   0,   0,   0,   0],  # México
        [0,    35,   0,   0,   0,   0,  40,   0,   0,   0,   0,   0,  35,   0,   0,  70,   0,   0,   0,   0],  # Indonesia
        [0,     0,  45,   0,  30,  40,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  80,  35,   0,   0],  # Países Bajos
        [0,     0,  35,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  30,  85,   0,   0],  # Suiza
        [0,    60,   0,   0,   0,   0,   0,   0,   0,   0,   0,  65,   0,   0,   0,   0,   0,   0,  90,   0],  # Arabia Saudita
        [0,     0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  75]   # Turquía
    ])

    country_names = [
        'USA', 'China', 'Alemania', 'Japón', 'Reino Unido',
        'Francia', 'India', 'Italia', 'Brasil', 'Canadá',
        'Corea Sur', 'Rusia', 'Australia', 'España', 'México',
        'Indonesia', 'Países Bajos', 'Suiza', 'Arabia Saudita', 'Turquía'
    ]

    # Ejecutar el análisis
    results = analyze_dependencies(X, country_names)


Top 10 dependencias más fuertes:
Importador | Exportador | Directa   | Indirecta | Total     | Long. Óptima
---------------------------------------------------------------------------
Arabia Saudita | Rusia      |   0.4375 |    0.0000 |   0.4375 |            2
Rusia      | Arabia Saudita |   0.4194 |    0.0000 |   0.4194 |            2
Canadá     | USA        |   0.3333 |    0.0456 |   0.3789 |            3
Japón      | China      |   0.3182 |    0.0574 |   0.3756 |            3
Corea Sur  | Japón      |   0.3333 |    0.0317 |   0.3650 |            3
Italia     | Alemania   |   0.2500 |    0.0885 |   0.3385 |            4
India      | China      |   0.2941 |    0.0392 |   0.3333 |            3
Francia    | Alemania   |   0.2063 |    0.1190 |   0.3254 |            3
Japón      | Corea Sur  |   0.2955 |    0.0277 |   0.3231 |            3
España     | Alemania   |   0.2195 |    0.0966 |   0.3161 |            4
Suiza      | Alemania   |   0.2500 |    0.0611 |   0.3111 |            4
País

In [87]:
denominators

array([253, 195, 229, 175, 209, 209, 146, 147, 144, 190])

In [ ]:
flows = [X_clean[path[i], path[i+1]] for i in range(len(path)-1)]